# Unsupervised Anomaly Detection on fastMRI

In [30]:
import pytorch_lightning as pl
import yaml

from trainer.trainer import AutoencoderModel

# autoreload imported modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the config

In [31]:
with open('./configs/autoencoder_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Reproducibility
pl.seed_everything(config['seed'])

Global seed set to 42


42

## Load and visualize the data

In [36]:
import torch


class DummyDataset(torch.utils.data.Dataset):
    def __init__(self):
        super(DummyDataset, self).__init__()
        self.data = torch.randn(100, 1, 128, 128)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], 0


train_dataset = DummyDataset()
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=config['batch_size'], shuffle=True)


## Prepare model

In [37]:
# Init model
model = AutoencoderModel(config)

max_epochs = config['num_epochs']
trainer = pl.Trainer(max_epochs=max_epochs)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [38]:
config

{'seed': 42, 'lr': 0.001, 'num_epochs': 10, 'batch_size': 32}

## Run training

In [39]:
# Train the model
trainer.fit(model, train_dataloaders=train_dataloader)

You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 105 K 
1 | decoder | Sequential | 105 K 
---------------------------------------
210 K     Trainable params
0         Non-trainable params
210 K     Total params
0.841     Total estimated model params size (MB)


{'seed': 42, 'lr': 0.001, 'num_epochs': 10, 'batch_size': 32}
Epoch 9: 100%|██████████| 4/4 [00:00<00:00, 163.00it/s, v_num=1]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 4/4 [00:00<00:00, 110.11it/s, v_num=1]


## Evaluation

In [ ]:
# TODO